In [ ]:
# imports
import numpy as np
import cuqi
import sys
import matplotlib.pyplot as plt
from cuqi.distribution import Gaussian, JointDistribution, GMRF, Gamma
from cuqi.geometry import Continuous2D
from cuqi.pde import TimeDependentLinearPDE
from cuqi.model import PDEModel
from copy import deepcopy
from custom_distribution import MyDistribution
from advection_diffusion_inference_utils import parse_commandline_args,\
    read_data_files,\
    create_domain_geometry,\
    create_PDE_form,\
    create_prior_distribution,\
    create_exact_solution_and_data,\
    set_the_noise_std,\
    sample_the_posterior,\
    create_experiment_tag,\
    plot_experiment,\
    save_experiment_data,\
    Args,\
    build_grids,\
    create_time_steps,\
    plot_time_series

In [ ]:
args = Args()
noise_level_list= ["fromDataVar" , "fromDataAvg", "avgOverTime", 0.1, 0.2]
args.noise_level = noise_level_list[1]
args.animal = 'm1'
args.ear = 'l'
args.num_ST = 0
args.inference_type = 'heterogeneous'
args.unknown_par_type = 'custom_1'
#args.unknown_par_value = ['m1:l:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5@../../../Collab-BrainEfflux-Data/April_2x_2024_b']
args.version = 'v200824_temp'

tag = create_experiment_tag(args)
print(tag)

In [ ]:
times, locations, real_data, real_std_data = read_data_files(args)
# The left boundary condition is given by the data  
real_bc = abs(real_data.reshape([len(locations), len(times)])[0,:])
#real_bc_r = abs(real_data.reshape([len(locations), len(times)])[-1,:])

In [ ]:
#%% STEP 4: Create the PDE grid and coefficients grid
#----------------------------------------------------
# PDE and coefficients grids
L = locations[-1]*1.3
coarsening_factor = 5
n_grid_c = 20
grid, grid_c, grid_c_fine, h, n_grid = build_grids(L, coarsening_factor, n_grid_c)

#%% STEP 5: Create the PDE time steps array
#------------------------------------------
tau_max = 30*60+10 # Final time in sec
cfl = 4 # The cfl condition to have a stable solution
         # the method is implicit, we can choose relatively large time steps 
tau = create_time_steps(h, cfl, tau_max)

#%% STEP 6: Create the domain geometry
#-------------------------------------
G_c = create_domain_geometry(grid, args.inference_type)

# STEP 7: Create the PDE form
#----------------------------
PDE_form = create_PDE_form(real_bc, grid, grid_c, grid_c_fine, n_grid, h, times,
                           args.inference_type)

# STEP 8: Create the CUQIpy PDE object
#-------------------------------------
PDE = TimeDependentLinearPDE(PDE_form,
                             tau,
                             grid_sol=grid,
                             method='backward_euler',
                             time_obs='all')#, 
                             #grid_obs=locations,
                             #time_obs=times)

# STEP 9: Create the range geometry
#----------------------------------
G_cont2D = Continuous2D((n_grid,len(PDE.time_steps)))

# STEP 10: Create the CUQIpy PDE model
#-------------------------------------
A = PDEModel(PDE, range_geometry=G_cont2D, domain_geometry=G_c)

In [ ]:
PDE.assemble(np.ones(n_grid+1))
temp_sol, _ = PDE.solve()

temp_sol.shape
n_grid

In [ ]:
PDE.time_steps

In [ ]:
## Differential operator (varying in space diffusion coefficient case)
Dx = - np.diag(np.ones(n_grid), 0)+ np.diag(np.ones(n_grid-1), 1) 
vec = np.zeros(n_grid)
vec[0] = 1
Dx = np.concatenate([vec.reshape([1, -1]), Dx], axis=0)
Dx /= h # FD derivative matrix


class MyModel:
    def __init__(self, PDE):
        self.PDE = PDE

    def grad(self, direc, wrt):
        # call the adjoint
        self.adjoint(direc, wrt)
        D_c_var = lambda c: -  Dx.T @ np.diag(2*c) @ Dx
        D_c_var_c = D_c_var(wrt)
        grad = np.zeros(len(wrt)-1)
        for i in range(len(times)):
            print("D_c_var_c.shape",D_c_var_c.shape)
            print("np.diag(self.u[:,i]).shape",np.diag(self.u[:,i]).shape)
            grad += (self.PDE.time_steps[-2]- self.PDE.time_steps[-3])*(D_c_var_c @ np.diag( self.u[:,i] ) )@ self.v[:,-i]
        return grad
    
    def adjoint(self, direc, wrt):
        # fill with zeros the direc 
        PDE_form_adjoint = create_PDE_form(real_bc, grid, grid_c, grid_c_fine, n_grid, h, times,
                           args.inference_type, adjoint=True, rhs_vec=-direc.reshape([n_grid, len(PDE.time_steps)]))
        PDE_adjoint = TimeDependentLinearPDE(PDE_form_adjoint,
                             tau,
                             grid_sol=grid,
                             method='backward_euler', 
                             #grid_obs=locations,
                             time_obs='all'
                             )
        PDE_adjoint.assemble(wrt)
        self.v, _ = PDE_adjoint.solve()
        # flip the adjoint solution
        


    def forward(self, x):
        # call the forward
        self.PDE.assemble(x)
        self.u, _ = self.PDE.solve()
        return self.u
    
model_obj = MyModel(PDE)

cuqi_model = cuqi.model.Model(forward=model_obj.forward, gradient=model_obj.grad, domain_geometry=G_c, range_geometry=G_cont2D)

        

In [ ]:
cuqi_model.forward(np.ones(n_grid+1))
G_c.par_dim
n_grid+1

In [ ]:
x = cuqi.distribution.GMRF(np.ones(G_c.par_dim)*np.sqrt(100), 2, geometry=G_c, bc_type='neumann')

y = Gaussian(cuqi_model(x), 0.1)

In [ ]:
ly = y.to_likelihood(data = temp_sol.flatten())
#test_val = 2+np.abs(np.random.randn(G_c.par_dim+1))*10
test_val = np.ones(G_c.par_dim+1)*np.sqrt(150)
grad = ly.gradient(test_val)

In [ ]:
# finie difference gradient
eps = 1e-5
from scipy.optimize import approx_fprime
def fun(x):
    return ly.logd(x)
grad_fd = approx_fprime(test_val, fun, eps)


In [ ]:
#sqrt_a =0.245 #np.sqrt(0.9)
#x_true, exact_data = create_exact_solution_and_data(A, args.unknown_par_type, args.unknown_par_value, a=sqrt_a, grid_c=grid_c)
#plot_time_series(times, locations, exact_data.reshape([len(locations), len(times)]))


In [ ]:
plt.plot(grad, 'r', label='AD')
plt.plot(grad_fd, 'b', label='FD')
plt.legend()


In [ ]:
im = plt.imshow(model_obj.v)
plt.colorbar(im)